In [1]:
import csv
from pprint import pprint
import re
import numpy as np
from tqdm import tqdm

In [2]:
migraine_file_names = ('migraine.com.csv','reddis_migraine_posts.csv')

In [3]:
posts_and_comments = []
header = []
read_header = False
for file in migraine_file_names:
    header = []
    read_header = False
    with open(f'data/{file}', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for entry in csv_reader:
            if not read_header:
                header = entry
                read_header = True
            else:
                posts_and_comments.append({
                    'Type': entry[header.index('Type')],
                    'Parent': entry[header.index('Parent')],
                    'Author': entry[header.index('Author')],
                    'Text': entry[header.index('Text')],
                    'Title': entry[header.index('Title')]
                })

print(len(posts_and_comments))


130066


In [4]:
pprint(posts_and_comments[120000])

{'Author': 'kimh12',
 'Parent': 'lnxj1w',
 'Text': 'I had none in the beginning. After being on it several years I '
         'started having word finding issues and had to write things down '
         'immediately or I’d forget them.',
 'Title': 'Topamax: how much "brain fog" are we talking about?',
 'Type': 'C'}


In [5]:
def pipe(functions):
    def run_pipe(input):
        result = input
        for function in functions:
            result = function(result)
        return result
    return run_pipe

In [6]:
def find_speaker_gender(postobj):
    text = postobj.get('Text')
    if text:
        re.search


In [7]:
test_m1 = "I am married and my Wife and I...."
test_m2 = "Me and my girlfriend went somewhere"
test_m3 = "Hello, me 44m and have migraines"
test_m4 = "Hello, me 44(m) and have migraines"
test_m5 = "Hello, me 44 (M) and have migraines"
test_m6 = "Hello, me 44 male and have migraines"
test_m7 = "Hello, I am male 44 and have migraines"

test_f1 = "Me and my husband have a car."
test_f2 = "Something I am currently pregnant and so on"
test_f3 = "Something I am pregnant and so on"
test_f4 = "Something I'm pregnant and so on"
test_f5 = "I have had menstruation related migraine"
test_f6 = "Me and my boyfriend went somewhere"
test_f7 = "Hello, me 44f and have migraines"
test_f8 = "Hello, me 44(f) and have migraines"
test_f9 = "Hello, me 44 (F) and have migraines"
test_f10 = "Hello, me 44 female and have migraines"
test_f11 = "Hello, I am female 44 and have migraines"

is_she = all([
    re.search('my\s+husband', test_f1, re.IGNORECASE),
    re.search('I( am|\'m)\s.*pregnant', test_f2, re.IGNORECASE),
    re.search('I( am|\'m)\s.*pregnant', test_f3, re.IGNORECASE),
    re.search('I( am|\'m)\s.*pregnant', test_f4, re.IGNORECASE),
    re.search('I\s.*menstruation', test_f5, re.IGNORECASE),
    re.search('my\s.*boyfriend', test_f6, re.IGNORECASE),
    re.search('\s[0-9][0-9](f|\(f\)|\s\(f\))', test_f7, re.IGNORECASE),
    re.search('\s[0-9][0-9](f|\(f\)|\s\(f\))', test_f8, re.IGNORECASE),
    re.search('\s[0-9][0-9](f|\(f\)|\s\(f\))', test_f9, re.IGNORECASE),
    re.search('\s[0-9][0-9].*female', test_f10, re.IGNORECASE),
    re.search('female.*[0-9][0-9]', test_f11, re.IGNORECASE)
])

assert(is_she)

is_he = all([
    re.search('my\s+wife', test_m1, re.IGNORECASE),
    re.search('my\s.*girlfriend', test_m2, re.IGNORECASE),
    re.search('\s[0-9][0-9](m\s|\(m\)|\s\(m\))', test_m3, re.IGNORECASE),
    re.search('\s[0-9][0-9](m\s|\(m\)|\s\(m\))', test_m4, re.IGNORECASE),
    re.search('\s[0-9][0-9](m\s|\(m\)|\s\(m\))', test_m5, re.IGNORECASE),
    re.search('\s[0-9][0-9].*male', test_m6, re.IGNORECASE),
    re.search('male.*[0-9][0-9]', test_m7, re.IGNORECASE)
])

assert(is_he)

In [8]:
female_matchers = [
        'my\s+husband',
        'I( am|\'m)\s.*pregnant',
        'I\s.*menstruation',
        'my\s.*boyfriend',
        '\s[0-9][0-9](f|\(f\)|\s\(f\))',
        '\s[0-9][0-9].*female',
        'female.*[0-9][0-9]',
    ]

male_matchers = [
    'my\s+wife',
    'my\s.*girlfriend',
    '\s[0-9][0-9](m\s|\(m\)|\s\(m\))',
    '\s[0-9][0-9].*male',
    'male.*[0-9][0-9]',
]

def check_gender(matchers):
    def check_text(text):
        return any([re.search(matcher, text, re.IGNORECASE) for matcher in matchers])
    return check_text

In [9]:
# test gender checker

female_texts = [
    test_f1,
    test_f2,
    test_f3,
    test_f4,
    test_f5,
    test_f6,
    test_f7,
    test_f8,
    test_f9,
    test_f10,
    test_f11,
]

male_texts = [
    test_m1,
    test_m2,
    test_m3,
    test_m4,
    test_m5,
    test_m6,
    test_m1,
]

female_matcher = check_gender(female_matchers)
male_matcher = check_gender(male_matchers)


assert(all([female_matcher(text) for text in female_texts]))
assert(all([male_matcher(text) for text in male_texts]))

In [10]:
identified_posts = []

for entry in posts_and_comments:
    gender = None

    if male_matcher(entry['Text']):
        gender = 'male'

    if female_matcher(entry['Text']):
        gender = 'female'

    if gender:
        identified_posts.append({
            'Author': entry['Author'],
            'Text': entry['Text'],
            'Parent': entry['Parent'],
            'Type': entry['Type'],
            'Gender': gender
        })

len(identified_posts)

2728

In [11]:
identified_posts[-10:]

[{'Author': 'superchiller',
  'Text': "Botox didn't work for my wife either, now she's on Emgality (a CGRP) and it has definitely helped, but not eliminated her migraines.",
  'Parent': 'l0klaq',
  'Type': 'C',
  'Gender': 'male'},
 {'Author': 'vap0urw3ed',
  'Text': "everytime i have a migraine i got an aura before, it's absolutely agonisingly bad. even if i take my medication in time to prevent it the aura stays for a good 15m or so.",
  'Parent': 'l0hhl0',
  'Type': 'C',
  'Gender': 'male'},
 {'Author': 'strega42',
  'Text': 'Migraineurs do often have much higher pain tolerances when it comes to chronic pain and head pain. This doesn\'t necessarily equate to higher pain tolerances of pain of different character, like kidney stones or abdominal pain, though.  \n\nAnd my husband has had career military combat veterans express surprise at how "hard core" he is when they witness him work through a migraine.  It\'s not just the headache; it is the inability to speak clearly/find words, t

In [12]:
# Create look up dict for auhtor's gender
author_gender_lookup = {}

for entry in identified_posts:
    if entry.get('Gender'):
        author_gender_lookup[entry['Author']] = entry['Gender']
print(len(author_gender_lookup))

1802


In [13]:
author_gender_lookup

{' jpp1223': 'male',
 ' fanghorn': 'male',
 ' nikkig127': 'female',
 ' dinga5': 'female',
 ' ngrassi': 'male',
 ' waggie': 'female',
 ' juliedavenport': 'female',
 ' tinuschka': 'female',
 ' mjking': 'male',
 ' ahnonnymust': 'female',
 ' mindjacked': 'female',
 ' teehar': 'female',
 ' mom46': 'female',
 ' stashharding': 'female',
 ' rarejules': 'female',
 ' hopelesssusie': 'female',
 ' pigen51': 'female',
 ' amyo': 'female',
 ' janeb': 'female',
 ' cforn': 'male',
 ' savta45': 'female',
 ' robynm67': 'female',
 ' Cheryl Picerno': 'female',
 ' Drjoec': 'male',
 ' ej96': 'male',
 ' lcantrell0428': 'female',
 ' amp254': 'female',
 ' ashleyheather': 'female',
 ' Ajdela': 'female',
 ' kattlyn': 'female',
 ' spacey-lacey': 'female',
 ' libra71': 'female',
 ' mello': 'female',
 ' bethrs': 'female',
 ' mjfmsc': 'female',
 ' gailmuzz': 'female',
 ' danadibs1979': 'female',
 ' texaslex': 'male',
 ' pennyd': 'female',
 ' mindy2496': 'male',
 ' chrisq': 'female',
 ' jenniedoc': 'female',
 ' migrai

In [14]:
for entry in posts_and_comments:
    if author_gender_lookup.get(entry['Author']):
        entry['Gender'] = author_gender_lookup[entry['Author']]
    else:
        entry['Gender'] = 'unknown'


In [15]:
count_f = 0
count_m = 0
count_u = 0

for entry in posts_and_comments:
    if entry['Gender'] == 'male':
        count_m += 1
    
    if entry['Gender'] == 'female':
        count_f += 1

    if entry['Gender'] == 'unknown':
        count_u += 1

print(f'male: {count_m}, female: {count_f}, unknown: {count_u}')

male: 5923, female: 32090, unknown: 92053


Find gender by username, cross reference to the above finds.

In [16]:
#import sys
#!{sys.executable} -m pip install gender_guesser
import gender_guesser.detector as gender 

In [66]:
#Test gender tool
d = gender.Detector()
nameall = ' Irrationally_Virgo' #nikki #rebecca
nameall = nameall.split()
name = nameall[0].capitalize()
sex = d.get_gender(name)
print(sex)
while sex == 'unknown':
    name = name[1:].capitalize()
    if not name:
        break
    sex = d.get_gender(name)
    print(name,sex)

unknown
Rrationally_virgo unknown
Rationally_virgo unknown
Ationally_virgo unknown
Tionally_virgo unknown
Ionally_virgo unknown
Onally_virgo unknown
Nally_virgo unknown
Ally_virgo unknown
Lly_virgo unknown
Ly_virgo unknown
Y_virgo unknown
_virgo unknown
Virgo male


In [18]:
gender_names = {}
d = gender.Detector()

count_fa = 0
count_ma = 0
count_ua = 0
female_pre = ('Miss','Mrs','Ms')
male_pre = ('Mr')

for entry in posts_and_comments:

    nameall = entry['Author'].split()
    
    try:
        name = nameall[0].capitalize()
    except:
        continue
    sex = d.get_gender(name)
    #remove characters at end of name to see if name is inside
    while sex == 'unknown':
        name = name[:-1].capitalize()
        if not name:
            break
        sex = d.get_gender(name)
    #try from other end:
    name = nameall[0]
    while sex == 'unknown':
        name = name[1:].capitalize()
        if not name:
            break
        sex = d.get_gender(name)
    #try prefixes
    if sex == 'unknown':
        name = nameall[0].capitalize()
        if name.startswith(female_pre):
            sex = 'female'
        elif name.startswith(male_pre):
            sex = 'male'
    
    if sex == 'male':  
        gender_names[entry['Author']] = sex
        count_ma += 1
    elif sex == 'female':
        gender_names[entry['Author']] = sex
        count_fa += 1
    else:
        gender_names[entry['Author']] = 'unknown'
        count_ua += 1

print(f'male: {count_ma}, female: {count_fa}, unknown: {count_ua}')
print(len(gender_names))


male: 24714, female: 36251, unknown: 68682
20821


In [61]:
#Cross Reference to text Regex
same = 0
diff = 0
userid = 0
body = 0
differs = []
ident = ['male','female']
namesused = []
for entry in posts_and_comments:
    if entry['Author'] in gender_names:
        if gender_names[entry['Author']] == entry['Gender']:
            same += 1
        elif gender_names[entry['Author']] != entry['Gender']:
            diff += 1   
            if gender_names[entry['Author']] in ident and entry['Gender'] in ident:
                if entry['Author'] not in namesused:
                    namesused.append(entry['Author'])
                    differs.append([entry,gender_names[entry['Author']]])
                #Check if one method says male and the other female, or vice versa
            elif entry['Gender'] not in ident:
                #Update main data source if unknown
                entry['Gender'] = gender_names[entry['Author']]
print('Same gender found:',same,'| Different gender found:',diff)
print(len(differs))
pprint(differs[-10:])

Same gender found: 106378 | Different gender found: 23269
288
[[{'Age': 0,
   'Author': 'greenbluewhite',
   'Dosage': 'unknown',
   'Gender': 'male',
   'Medicine': 'unknown',
   'Parent': 'lo5m1r',
   'Qty': 'unknown',
   'Suicidal': 'no',
   'Text': 'Google "ocular migraine".  My wife thinks she has this.  Symptoms '
           'go away in under an hour usually. No associated headache.',
   'Title': 'Has anyone noticed shimmering vision\\pulsating vision\\Sensation '
            'of heat waves when you were outside ?',
   'Type': 'C'},
  'female'],
 [{'Age': 0,
   'Author': 'One_Argument2921',
   'Dosage': 'unknown',
   'Gender': 'male',
   'Medicine': 'unknown',
   'Parent': 'lmtkkk',
   'Qty': 'unknown',
   'Suicidal': 'no',
   'Text': 'So for the past year now I have had this constant pressure in my '
           'head around my temples, and behind my eyes. This has come with '
           'extreme brain fog and my vision also blurs sometimes. For a year '
           'and some chan

In [50]:
count_f = 0
count_m = 0
count_u = 0

for entry in posts_and_comments:
    if entry['Gender'] == 'male':
        count_m += 1
    
    if entry['Gender'] == 'female':
        count_f += 1

    if entry['Gender'] == 'unknown':
        count_u += 1

print(f'male: {count_m}, female: {count_f}, unknown: {count_u}')

male: 23920, female: 54300, unknown: 51846


# Find Information about Drugs


In [21]:
drug_list = [
    'Amitriptyline',
    'Elavil',
    'Divalproex',
    'Depakote',
    'Eletriptan',
    'Relpax',
    'triptan',
    'Metoprolol',
    'Lopressor',
    'Toprol',
    'Propranolol',
    'Inderal',
    'beta blocker',
    'Rizatriptan',
    'Maxalt',
    'Sumatriptan',
    'Imitrex',
    'Topiramate',
    'Topamax',
    'Trokendi',
    'Venlafaxine',
    'Effexor',
    'Zolmitriptan',
    'Zomig',
    'OnabotulinumtoxinA',
    'Botox',
    'Erenumab',
    'Aimovig',
    'CGRP',
    'Nurtec',  # found in the subreddit post
    'Topomax'  # popular misspelling of Topamax,
    'nortiptyline',  # found in the subreddit post
    'metoclopramide',  # found in the subreddit post
    'caffeine pill',  # found in the subreddit posts_and_comments
    'naproxen'
]

In [22]:
test_med1 = "I take 75mg topamax daily"

r = re.search('([0-9]+mg|[0-9]+\smg)', test_med1)

has_meds = []

for entry in posts_and_comments:
    r = re.search('([0-9]+mg|[0-9]+\smg)', entry['Text'])
    if r:
        has_meds.append(entry)

len(has_meds)

4319

In [23]:
# for m in has_meds:
#     print(m['Text'])
#     print('--------------------------------------------------')

## Medicine Patterns

Manually sampling above output we found following patterns to base our regex expressions on:

- 75mg topamax daily
- Recently been prescribed 25mg topiramate to take one a day before bed
- I'm on 50mg and feel no effect at all
- I'm on 50mg and dont feel completely wrecked the next day. But I also have a sleep disorder that benefits from amitriptyline making me tired at night
- I’m currently on daily 40mg of Propranolol
- my current amitryptiline dose (50mg)
- prescribed me 50mg amitriptyline
- I take 2 x 600 mg caps of magnesium
- I take 10mg of edible Delta 8
- I was on 10mg which worked great for 6 months
- Now I take 30 mg
- It's a combination of sumatriptan 85 mg and naproxen sodium 500 mg
- I use 50 mg with a triptan
- I currently take 75mg daily
- I'm at 20mg 3x a day now
- I started Aimovig in July 2018 at the 70mg dose
- Moved up to the 140 about 9 months later
- I take 50mg daily
- She said 875mg of Amoxicillin
- I take a total of 2400mg/day: 900mg/600mg/900mg.
- it 80mg twice a day
- Years ago i took 900mg three times a day
- 300mg 3x a day
- sulfate 325 mg twice daily
- I took .5 mg of Xanax
- my doc prescribed me 10mg of amitriptyline nightly
- my dose is 2x 25mg
- Topamax took about a month for me (at 50 mg)
- Mine comes in 2.5 or 5mg.
- but I take 250mg/day


In [24]:
test_texts = [
    '75mg topamax daily',
    'Recently been prescribed 25mg topiramate to take one a day before bed',
    "I'm on 50mg and feel no effect at all",
    "I'm on 50mg and dont feel completely wrecked the next day. But I also have a sleep disorder that benefits from amitriptyline making me tired at night",
    'I’m currently on daily 40mg of Propranolol',
    'my current amitryptiline dose (50mg)',
    'prescribed me 50mg amitriptyline',
    'I take 2 x 600 mg caps of magnesium',
    'I take 10mg of edible Delta 8',
    'I was on 10mg which worked great for 6 months',
    'Now I take 30 mg',
    'It\'s a combination of sumatriptan 85 mg and naproxen sodium 500 mg',
    'I use 50 mg with a triptan',
    'I currently take 75mg daily',
    'I\'m at 20mg 3x a day now',
    'I started Aimovig in July 2018 at the 70mg dose',
    'Moved up to the 140 about 9 months later',
    'I take 50mg daily',
    'She said 875mg of Amoxicillin',
    'I take a total of 2400mg/day: 900mg/600mg/900mg.',
    'it 80mg twice a day',
    'Years ago i took 900mg three times a day',
    '300mg 3x a day',
    'sulfate 325 mg twice daily',
    'I took .5 mg of Xanax',
    'my doc prescribed me 10mg of amitriptyline nightly',
    'my dose is 2x 25mg',
    'Topamax took about a month for me (at 50 mg)',
    'Mine comes in 2.5 or 5mg.',
    'but I take 250mg/day'
]

results = []

rl = [
    {
        'regex': re.compile('([0-9]x).*([0-9]+mg|[0-9]+\smg)', flags=re.IGNORECASE),
        'dosage_group': 2,
        'qty_group': 1
    },
    {
        'regex': re.compile('([0-9]+mg|[0-9]+\smg).*([0-9]x)', flags=re.IGNORECASE),
        'dosage_group': 1,
        'qty_group': 2
    },
    {
        'regex': re.compile('([0-9]+mg|[0-9]+\smg).*(three times a day|four times a day|twice a day|one a day|twice daily)', flags=re.IGNORECASE),
        'dosage_group': 1,
        'qty_group': 2
    },
    {
        'regex': re.compile('([0-9]+mg|[0-9]+\smg).*(nightly|daily|dose|day)', flags=re.IGNORECASE),
        'dosage_group': 1,
        'qty_group': 2
    }
]

def normalize_qty(qty_text):
    if qty_text == 'daily' or qty_text == 'dose' or qty_text == 'day' or qty_text == 'one a day' or qty_text == '1x':
        return '1x'

    if qty_text == 'twice daily' or qty_text == 'twice a day':
        return '2x'

    if qty_text == 'three times a day':
        return '3x'

    return qty_text

def find_match(text):
    for r in rl:
        m = r['regex'].search(text)
        if m:
            return m, r
    return None, None

for text in test_texts:
    rr, reg = find_match(text)
    if rr:
        results.append({
            'Text': text,
            'Group0': rr.group(0),
            'Dosage': rr.group(reg['dosage_group']),
            'Times': normalize_qty(rr.group(reg['qty_group']))
        })

results

[{'Text': '75mg topamax daily',
  'Group0': '75mg topamax daily',
  'Dosage': '75mg',
  'Times': '1x'},
 {'Text': 'Recently been prescribed 25mg topiramate to take one a day before bed',
  'Group0': '25mg topiramate to take one a day',
  'Dosage': '25mg',
  'Times': '1x'},
 {'Text': "I'm on 50mg and dont feel completely wrecked the next day. But I also have a sleep disorder that benefits from amitriptyline making me tired at night",
  'Group0': '50mg and dont feel completely wrecked the next day',
  'Dosage': '50mg',
  'Times': '1x'},
 {'Text': 'I currently take 75mg daily',
  'Group0': '75mg daily',
  'Dosage': '75mg',
  'Times': '1x'},
 {'Text': "I'm at 20mg 3x a day now",
  'Group0': '20mg 3x',
  'Dosage': '20mg',
  'Times': '3x'},
 {'Text': 'I started Aimovig in July 2018 at the 70mg dose',
  'Group0': '70mg dose',
  'Dosage': '70mg',
  'Times': '1x'},
 {'Text': 'I take 50mg daily',
  'Group0': '50mg daily',
  'Dosage': '50mg',
  'Times': '1x'},
 {'Text': 'I take a total of 2400mg/

In [25]:
posts_and_comments[600]

{'Type': 'C',
 'Parent': '17.9',
 'Author': ' Nancy Harris Bonk',
 'Text': "Thanks awful, Tracy-wol .  I'm sorry you aren't getting the care you need. Is there any way you can see a doctor who is an expert in treating migraine and headache disease? Or at least someone more knowledgeable about it? It's appalling we are left to suffer. No one deserves that.\nPlease keep me posted and I hope you have a low pain day, Nancy Harris Bonk, Patient Advocate/Moderator ",
 'Title': ' New to Migraines  ',
 'Gender': 'female'}

In [26]:
def find_medicine_name(entry):
    meds_matched = []
    for drug in drug_list:
        meds_matched.extend(re.findall(drug, text, re.IGNORECASE))
    return meds_matched

In [27]:
def find_dosage(text):
    rr, reg = find_match(text)
    if rr:
        return (rr.group(reg['dosage_group']), normalize_qty(rr.group(reg['qty_group'])))
    return (None, None)

In [28]:
def find_parent_post(id):
    for entry in posts_and_comments:
        if entry['Parent'] == id and entry['Type'] == 'P':
            return entry
    return None

In [29]:
author_meds = {}

for entry in posts_and_comments:
    text = entry['Text']
    dosage, qty = find_dosage(text)
    if dosage:
        m = find_medicine_name(text)
        if len(m) == 0 and entry['Type'] == 'C':
            # if no medicine name in comment, check parent post
            parent_entry = find_parent_post(entry['Parent'])
            if parent_entry:
                m = find_medicine_name(parent_entry['Text']) 
        if m:
            author_meds[entry['Author']] = {
                'medicine': m[0],
                'dosage': dosage,
                'qty': qty
            }
len(author_meds)

1019

In [30]:
entries_with_meds_count = 0
entries_with_meds_and_gender = 0

for entry in posts_and_comments:
    auth_match = author_meds.get(entry['Author'])
    if auth_match:
        entry['Medicine'] = auth_match['medicine']
        entry['Dosage'] = auth_match['dosage']
        entry['Qty'] = auth_match['qty']
        entries_with_meds_count += 1
        if entry.get('Gender') != 'unknown':
            entries_with_meds_and_gender += 1
    else:
        entry['Medicine'] = 'unknown'
        entry['Dosage'] = 'unknown'
        entry['Qty'] = 'unknown'

(entries_with_meds_count, entries_with_meds_and_gender)

(27566, 17245)

# Suicidal Thoughts

This section attempts to identify all of the authors that had suicidal thoughts.

Following is the list of sample sentences talking about suicide and they will serve as a test sentences and patterns for creating regex expression to retrieve information on suicidal thoughts.

In [31]:
suicide_test_sentences = [
    'Had suicidal thoughts',
    'made me think a lot about suicide',
    'I still thought about suicide',
    'Suicide ideation',
    'suicidal ideations',
    'The “intrusive thoughts” and experiencing life far away'
    'I had felt suicidal',
    'feeling sad/suicidal',
    'it made me wildly suicidal',
    'I have been extremely suicidal',
    'i also have been extremely suicidal',
    'and then attempted suicide to stop the pain',
    'my near suicide attempt',
    'Depression and suicide thoughts are unbearable for me',
    'I was having suicidal thoughts',
    'I self harm and have suicidal thoughts',
    'just made me suicidal',
    'I was so suicidal',
    'made me legitimately suicidal',
    'Had suicidal thoughts',
    'I still thought about suicide',
    'made me think a lot about suicide'
]

neg_suicide_test_sentences = [
    'I am not suicidal',
    'I have never been suicidal'
]

In [32]:
positive_suicide_matchers = [
    re.compile('(am|have|had|felt|having|me|was|been|think|about|feeling).*(suicidal|suicide)', re.IGNORECASE),
    re.compile('(my near|made me|have been|thought about|).*(suicidal|suicide)', re.IGNORECASE)
]

negative_suicide_matchers = [
    re.compile('(am|have|had|felt|having|me|was|been|think|about|feeling) (not|never).*(suicidal|suicide)', re.IGNORECASE),
    re.compile('(my near|made me|have been|thought about|) (not|never).*(suicidal|suicide)', re.IGNORECASE)
]

In [33]:
def search_for_suicide(text):
    return any([matcher.search(text) for matcher in positive_suicide_matchers]) \
        and not any([matcher.search(text) for matcher in negative_suicide_matchers])


In [34]:
# Should be True
l = [search_for_suicide(text) for text in suicide_test_sentences]
print(l)
all(l)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


True

In [35]:
# Should be False
not_l = [search_for_suicide(text) for text in neg_suicide_test_sentences]
print(not_l)
any(not_l)

[False, False]


False

In [36]:
# TEST TEST - IGNORE
t1 = "that is suicide."
t2 = "that was suicidal."
t3 = "that is not suicidal."
t4 = "that was never suicide."

r = re.search('(is|was) (?!not|never).*(suicide|suicidal)', t1, re.IGNORECASE)
r

<re.Match object; span=(5, 15), match='is suicide'>

In [37]:
# Create Author index who had suicidal thoughts
author_suicide_lookup = {}

for entry in posts_and_comments:
    if search_for_suicide(entry['Text']):
        author_suicide_lookup[entry['Author']] = 'suicidal'
len(author_suicide_lookup)

226

In [38]:
# Now update all posts
total_suicidal = 0
for entry in posts_and_comments:
    if author_suicide_lookup.get(entry['Author']):
        entry['Suicidal'] = 'yes'
        total_suicidal += 1
    else:
        entry['Suicidal'] = 'no'
total_suicidal

14076

In [39]:
# Discover author's age


In [40]:
age_test_sentences = [
    "I'm 26",
    "I'm 9",
    "I am 26",
    "I'm in my 40's",
    "45 years old",
    "I am 20 (F)",
    "I am 28M",
    "I am now at 58",
    "I'm now 20",
    "40f"
]

no_age_sentences = [
    "I'm good",
    "I am bad",
    "I'm in my house",
    "years old",
    "I am (F)",
    "I am M",
    "I am now at the shop",
    "I'm now better",
    "is f"
]

In [41]:
age_matchers = [
    {'matcher': re.compile("I('m| am) ([0-9][0-9]*)", re.IGNORECASE), 'group': 2},
    {'matcher': re.compile("I('m| am) in my ([0-9][0-9]*)", re.IGNORECASE), 'group': 2},
    {'matcher': re.compile("([0-9][0-9]*) years old", re.IGNORECASE), 'group': 1},
    {'matcher': re.compile("I('m| am) now ([0-9][0-9]*)", re.IGNORECASE), 'group': 2},
    {'matcher': re.compile("I('m| am) now at ([0-9][0-9]*)", re.IGNORECASE), 'group': 2},
    {'matcher': re.compile("([0-9][0-9]*)(f|m)", re.IGNORECASE), 'group': 1},
    {'matcher': re.compile("([0-9][0-9]*) (f|m)", re.IGNORECASE), 'group': 1},
    {'matcher': re.compile("([0-9][0-9]*)\((f|m)\)", re.IGNORECASE), 'group': 1},
    {'matcher': re.compile("([0-9][0-9]*) \((f|m)\)", re.IGNORECASE), 'group': 1}
]

In [42]:
def find_age(text):
    for matcher in age_matchers:
        if (r := matcher['matcher'].search(text)):
            return int(r.group(matcher['group']))
    return 0

In [43]:
# Test regex

ages = [find_age(sentence) for sentence in age_test_sentences]
print(ages)
all(ages)

[26, 9, 26, 40, 45, 20, 28, 58, 20, 40]


True

In [44]:
ages = [find_age(sentence) for sentence in no_age_sentences]
print(ages)
all(ages)

[0, 0, 0, 0, 0, 0, 0, 0, 0]


False

In [45]:
# Create dictionary of authors and their ages
author_age = {}

for entry in posts_and_comments:
    if (age := find_age(entry['Text'])):
        author_age[entry['Author']] = age
len(author_age)

6936

In [46]:
# Update all entries with age
total_ages = 0

for entry in posts_and_comments:
    age = author_age.get(entry['Author'], 0)
    entry['Age'] = age
    if age:
        total_ages += 1

total_ages

87260

In [54]:
pprint(posts_and_comments[10080:10100])

[{'Age': 3,
  'Author': ' lupismoon',
  'Dosage': 'unknown',
  'Gender': 'unknown',
  'Medicine': 'unknown',
  'Parent': '841.37',
  'Qty': 'unknown',
  'Suicidal': 'no',
  'Text': 'Hi... new here. Never really talk about my pain- migraine or '
          'otherwise -except with my Sister-in-law and husband. Even then I '
          'don’t talk too much about it, I want to be normal, not pitied.  I '
          'have had my migraines under control for many years - ie: I know my '
          'triggers, relaxation, meditation, and the like. Migraine and I came '
          'to an understanding finally about 10 years ago...  Six months ago I '
          'started getting a nagging migraine off and on. Nothing I couldn’t '
          'deal with.  BANG! Started vomiting and seeing double amongst all '
          'the other "signs" about 3 months ago, two days later I was in the '
          'ER with a humdinger of a migraine - this one brought me to my '
          'knees...Now I live in constant fea